# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [1]:
def classifier(smoker, age, diet):
    if smoker == 'yes':
        if age < 29.5:
            return 'less'
        else:
            return 'more'
    else:
        if diet == 'good':
            return 'less'
        else:
            return 'more'
        
print(classifier('yes', 31, 'good'))

more


## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [2]:
test_list = []
test_file = open('health-test.txt')

for line in test_file:
    smoker, age, diet = line.split(',')
    test_list.append((smoker, int(age), diet.rstrip()))
    
for test in test_list:
    print(test)

('yes', 21, 'poor')
('no', 50, 'good')
('no', 23, 'good')
('yes', 45, 'poor')
('yes', 51, 'good')
('no', 60, 'good')
('no', 15, 'poor')
('no', 18, 'good')


## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more risk".

In [3]:
counter = 0

for test in test_list:
    if classifier(*test) == 'more':
        counter += 1
        
print(counter/len(test_list))

0.375


## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [4]:
train_list = []
train_file = open('health-train.txt')

for line in train_file:
    smoker, age, diet, label = line.split(',')
    train_list.append(((smoker, int(age), diet), label.rstrip()))
    
for train in train_list:
    print(train)

(('yes', 54, 'good'), 'less')
(('no', 55, 'good'), 'less')
(('no', 26, 'good'), 'less')
(('yes', 40, 'good'), 'more')
(('yes', 25, 'poor'), 'less')
(('no', 13, 'poor'), 'more')
(('no', 15, 'good'), 'less')
(('no', 50, 'poor'), 'more')
(('yes', 33, 'good'), 'more')
(('no', 35, 'good'), 'less')
(('no', 41, 'good'), 'less')
(('yes', 30, 'poor'), 'more')
(('no', 39, 'poor'), 'more')
(('no', 20, 'good'), 'less')
(('yes', 18, 'poor'), 'less')
(('yes', 55, 'good'), 'more')


## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [5]:
def d(a,b):
    return (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])

def nnclassifier(test_point, train_list):
    min_d = float('inf')
    for train in train_list:
        dist = d(test_point, train[0])
        if dist < min_d:
            min_d = dist
            nn = train
    return (test_point, nn[1])

print(nnclassifier(("yes", 31, "good"), train_list))

(('yes', 31, 'good'), 'more')


* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and with which probability it happens.

In [6]:
counter = 0
dis_list = []

for test in test_list:
    if classifier(*test) != nnclassifier(test, train_list)[1]:
        counter += 1
        dis_list.append(test)
        
print(dis_list)
print(counter/len(test_list))

[('yes', 51, 'good')]
0.125


One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [7]:
def dist(a, b):
    return (a[0]-b[0])**2 + ((a[1]-b[1])/50.0)**2 + (a[2]-b[2])**2

class NearestMeanClassifier:
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.
    def train(self,dataset):
        smoke_l = 0
        age_l = 0
        diet_l = 0
        
        smoke_m = 0
        age_m = 0
        diet_m = 0
        
        counter_l = 0
        counter_m = 0
        
        for item in dataset:
            if item[1] == 'less':
                smoke_l += 1 if item[0][0] == 'yes' else 0
                age_l += item[0][1]
                diet_l += 1 if item[0][2] == 'poor' else 0
                counter_l += 1
            else:
                smoke_m += 1 if item[0][0] == 'yes' else 0
                age_m += item[0][1]
                diet_m += 1 if item[0][2] == 'poor' else 0
                counter_m += 1
        
        self.less = (smoke_l/counter_l, age_l/counter_l, diet_l/counter_l)
        self.more = (smoke_m/counter_m, age_m/counter_m, diet_m/counter_m)
    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        x = (1 if x[0] == 'yes' else 0, x[1], 1 if x[2] == 'poor' else 0)
        l = dist(self.less, x)
        m = dist(self.more, x)
        
        if l < m:
            return 'less'
        else:
            return 'more'
        

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [8]:
c = NearestMeanClassifier()
c.train(train_list)

print(c.less)
print(c.more)

(0.3333333333333333, 32.111111111111114, 0.2222222222222222)
(0.5714285714285714, 37.142857142857146, 0.5714285714285714)


* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [9]:
agr_list = []

for test in test_list:
    if classifier(*test) == nnclassifier(test, train_list)[1] == c.predict(test):
        agr_list.append((test, classifier(*test)))
    
for item in agr_list:
    print(item)

(('no', 50, 'good'), 'less')
(('no', 23, 'good'), 'less')
(('yes', 45, 'poor'), 'more')
(('no', 60, 'good'), 'less')
(('no', 15, 'poor'), 'more')
(('no', 18, 'good'), 'less')
